In [21]:
### Solving x^5 - 2*y^5 = 1
### Method of Tzanakis and Weger


m=1
x = polygen(ZZ, 'x')
poly=x^5 - 2
deg=poly.degree()
K.<theta> = NumberField(poly)
UG = UnitGroup(K)
print(UG)
FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
rk=len(FU)
print(rk)
print(FU)
thetas=poly.roots(SR,multiplicities=False)
print(thetas)
norm_list=K.elements_of_norm(m)
print(norm_list)


### Ordering the roots and embeddings so that the real ones are in front 
thetas=[(2^(1/5)),(2^(1/5))*e^((2*I*pi)/5),(2^(1/5))*e^((4*I*pi)/5),(2^(1/5))*e^(-(2*I*pi)/5),(2^(1/5))*e^(-(4*I*pi)/5)]
print(thetas)
from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]
print(emb)

Unit group with structure C2 x Z x Z of Number Field in theta with defining polynomial x^5 - 2
2
[theta^3 + theta^2 - 1, theta - 1]
[1/4*2^(1/5)*(sqrt(5) + I*sqrt(2*sqrt(5) + 10) - 1), -1/4*2^(1/5)*(sqrt(5) - I*sqrt(-2*sqrt(5) + 10) + 1), -1/4*2^(1/5)*(sqrt(5) + I*sqrt(-2*sqrt(5) + 10) + 1), 1/4*2^(1/5)*(sqrt(5) - I*sqrt(2*sqrt(5) + 10) - 1), 2^(1/5)]
[1]
[2^(1/5), 1/4*2^(1/5)*(sqrt(5) + I*sqrt(2*sqrt(5) + 10) - 1), -1/4*2^(1/5)*(sqrt(5) - I*sqrt(-2*sqrt(5) + 10) + 1), 1/4*2^(1/5)*(sqrt(5) - I*sqrt(2*sqrt(5) + 10) - 1), -1/4*2^(1/5)*(sqrt(5) + I*sqrt(-2*sqrt(5) + 10) + 1)]
[Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> 2^(1/5), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> 1/4*2^(1/5)*(sqrt(5) + I*sqrt(2*sqrt(5) + 10) - 1), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -

In [27]:
### computing c1
M=matrix(RR,deg,deg)
for i in range(deg):
    for j in range(deg):
        if j < i:
            M[i,j] = abs(thetas[i]-thetas[j])
        else:
            M[i,j] = abs(thetas[i])+abs(thetas[j])


c1=2/min(M.list())
print('c1=',c1)


### computing c2, c3
### might take long time, as there are deg^3 computations
L=[0 for i in range(deg)]
for i in range(deg):
    L[i]=max([abs((thetas[i]-thetas[j])/(thetas[i]-thetas[k])).n(prec=40) for j in range(deg) for k in range(deg) if j != i if k != j if k != i ])
c2=max(L)
print('c2=',c2)
c3=c1*c2
print('c3=',c3)


### computing c4, c5
MFU = matrix(CC, rk+1, rk, lambda i, j: ln(abs(emb[i](FU[j]).n(prec=20))))   ### Matrix of log of abs of fundamental units
V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### Inverse of matrix U_I as in the text
c4=1/(max([V[i].norm(Infinity) for i in range(rk+1)] ))
print('c4=',c4)
c5=c4/(deg-0.9)
print('c5=',c5)

c1= 1.48106908075834


c2= 1.6180339888
c3= 2.3964201124
c4= 0.44953561306936024
c5= 0.10964283245594153


In [28]:
### cases A,B 
### computing c6, c7 and the corresponding A1,A2


M=matrix(RR,deg,len(norm_list))
for i in range(deg):
    for j in range(len(norm_list)):
        M[i,j]=abs(emb[i](norm_list[j]).n(prec=20))^(-1)         
c6=max(M.list())
c7=min(M.list())
print('c6=',c6)
print('c7=',c7)
A1=ln(c6*abs(m))/(c4-(deg-1)*c5)
A1_min=ln(c6*abs(m))/(c4)
print('A1=',A1)
print('A1_min=',A1_min)
A2=ln(c7)/(c4-c5)
print('A2=',A2)

c6= 1.00000000000000
c7= 1.00000000000000
A1= 0.000000000000000
A1_min= 0.000000000000000
A2= 0.000000000000000


In [29]:
### Fucntion of modified height
### Input : X, a list of n non-integer elements
### Output: the modified heights of each element of X, and the index of [Q(X):Q]

def HeightMod(X):
    n=len(X)
    P=[(X[i]).minpoly() for i in range(n)]
    deg_global=(prod(P).splitting_field('x', simplify_all=True).polynomial()).degree()
    HM=[max( (P[i]).global_height(),  abs(ln(X[i]))/deg_global,   1/deg_global   )   for i in range(n)]
    return(HM,deg_global)

In [30]:
### Case C
### computing c8 and an initial bound on A

alpha2=(thetas[0]-thetas[1])/(thetas[2]-thetas[0])
unit1=emb[2](FU[0])/emb[1](FU[0])
unit2=emb[2](FU[1])/emb[1](FU[1])
result=HeightMod( [alpha2,unit1,unit2] )
heights=result[0]
deg_global=result[1]
hm0=max([abs(ln(-1))/deg_global, 1/deg_global])
c8=(18*factorial(5)*(4^5)*(256^6)*ln(64)*prod(heights)*hm0)
print("c8=%E" %c8)
A_0=(2/c5)*( ln(2*c3) +c8*ln(3/2) +c8*ln(c8/c5)  ).n(prec=16)
print('upper bound on A=',A_0)

c8=1.238095E+23
upper bound on A= 1.260e26


In [32]:
p1=alpha2.minpoly()
p2=unit1.minpoly()
p3=unit2.minpoly()
print(p1.global_height())
print(p2.global_height())
print(p3.global_height())
print(p1,p2,p3)

1.38629436111989
16.7408871031923
13.1168602630084
x^4 + 2*x^3 + 4*x^2 + 3*x + 1 x^20 - 20*x^19 + 390*x^18 - 1340*x^17 + 20845*x^16 + 47496*x^15 + 3054760*x^14 + 10721880*x^13 + 6951570*x^12 - 11074960*x^11 - 18641244*x^10 - 11074960*x^9 + 6951570*x^8 + 10721880*x^7 + 3054760*x^6 + 47496*x^5 + 20845*x^4 - 1340*x^3 + 390*x^2 - 20*x + 1 x^20 + 30*x^19 + 340*x^18 + 1710*x^17 + 3045*x^16 - 28004*x^15 + 109360*x^14 - 236220*x^13 + 394170*x^12 - 468060*x^11 + 497256*x^10 - 468060*x^9 + 394170*x^8 - 236220*x^7 + 109360*x^6 - 28004*x^5 + 3045*x^4 + 1710*x^3 + 340*x^2 + 30*x + 1


In [40]:
### Function of the LLL reduction step
### Input: alpha_0, ALPHA=(alpha_1,...,alpha_n),Z0,p_1,p_2
###      as in the text
### Output: the reduced upper bound


def LLLReduction(alpha0,ALPHA,Z0,p1,p2):           
    from sage.functions.log import logb
    C=10^((1.75*logb(Z0, 10).n(prec=40)).ceiling())
    n=len(ALPHA)
    M=matrix.identity(n)
    
    for i in range(n):
        M[n-2,i]=round(C*real_part(  ALPHA[i]  ).n(prec=40))
        M[n-1,i]=round(C*imag_part(  ALPHA[i]  ).n(prec=40))
        
    Y=zero_vector(n)
    Y[n-2]=round(C*real_part(  alpha0  ).n(prec=40))
    Y[n-1]=round(C*imag_part(  alpha0  ).n(prec=40))
    M=M.T
    MLLL=(M.LLL())
    MLLL=MLLL.T
    GS1,GS2=(MLLL).gram_schmidt()
    cc1=max([norm(MLLL.column(0))^2/ norm(GS1.column(i))^2 for i in range(n)])
    
    
    sigma=(MLLL).inverse()*Y
    sigma_dist_vec=[min([sigma[i]-floor(sigma[i]),ceil(sigma[i])-sigma[i]]) for i in range(n)]
    sigma_dist_vec=[x for x in sigma_dist_vec if x!= 0]
    sigma_dist=sigma_dist_vec[len(sigma_dist_vec)-1]
    
    L=cc1^(-1)*sigma_dist*norm(MLLL.column(0))^2
    SS=(n-2)*(Z0)^2
    TT=(1+n*Z0)/sqrt(2)
    
    upper_bound=floor(abs((1/p2)*(ln(C*p1)-ln(sqrt(L^2-SS)-TT) )))
    return(upper_bound)

In [41]:
alpha0=ln(-alpha2)
ALPHA=[unit1,unit2,2*pi*sqrt(-1)]
Z0=A_0
p1=2*c3
p2=c5
LLLReduction(alpha0,ALPHA,Z0,p1,p2)

374

In [35]:
### Reduce the bound after two rounds of LLL

alpha0=ln(-alpha2)
ALPHA=[unit1,unit2,2*pi*sqrt(-1)]
Z0=A_0
p1=2*c3
p2=c5
bound1=LLLReduction(alpha0,ALPHA,Z0,p1,p2)
print(bound1)
if bound1 >=50:
    bound2=LLLReduction(alpha0,ALPHA,bound1,p1,p2)
else:
    bound2=bound1
print(bound2)
XY=[]
for i in range(-bound2,bound2):
    for j in range(-bound2,bound2):
        for k in range(len(norm_list)):
            result=(norm_list[k])*(FU[0]^i)*(FU[1]^j)
            P=result.polynomial(var='t')
            if P.degree() <2:
                a0=abs(result[0])
                a1=abs(result[1])
                XY.append((a0,a1))
print(sorted(set(XY)))

374


21
[(1, 0), (1, 1)]


In [0]:
############
############
############

In [7]:
### Method of Bilu and Hanrot


CC500=ComplexField(500)
ComplexNumber=CC500
m=1
x = polygen(ZZ, 'x')
poly=x^5 - 2
deg=poly.degree()

K.<theta> = NumberField(poly) 
UG = UnitGroup(K)

FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
rk=len(FU)
thetas=poly.roots(SR,multiplicities=False)

norm_list=K.elements_of_norm(m)


### Ordering the roots and embeddings so that the real ones are in front, followed by distinct complex ones, then the conjugates 
    
real_roots=[]
comp_roots=[]
for i in thetas:
    if i.is_real() == True:
        real_roots.append(i)
    else:
        comp_roots.append(i)
r1=len(real_roots)
r2=(deg-r1)/2
if r1 < deg:
    ordered_roots=[0 for i in range(deg)]
for i in range(r1):
    ordered_roots[i]=real_roots[i]
index=r1
for i in range(2*r2):
    if comp_roots[i] not in ordered_roots :
        ordered_roots[index]=comp_roots[i]
        ordered_roots[index+r2]=comp_roots[i].conjugate()
        index=index+1
thetas=ordered_roots

from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]

MFU = matrix( CC500, rk+1, rk, lambda i, j: ln( abs(emb[i](FU[j])) ) )   ### Matrix of log of abs of fundamental units, we need high precision
print(MFU)

II=[]
for i in range(rk+1):
    II.append(  [i,[j for j in range(rk+1) if j!=i]]   )
    
print(II)
V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### V_I, inverse of matrix U_I as in the text
print(V)

[0.607166804969287243111216210942423429745767613669124828345498426234005680555640965563537466488382923837481269287694073588696895129859205134138535756335 -1.90583548813918861720241094731814479326955200520550104131253856654035214674360819543454132117215979160319148388914283285425952176047170158892277432330]
[ 1.19263777924128032831303931656542909914629876094995361237142508681628239104248931827227055635282384058744701448613723425492645657993813028251600520667 0.237984551611593668794698352463099882507971955293442026401155232195241999234952925912623075765162016196182395431324093164537403109593983952674604788989]
[-1.49622118172592394986864742203664081401918256778451602654417429993328523132030980105403928959701530250618764912998427104927490414486773284958527308483 0.714933192458000639806507121195972514126804047309308494255114051074934074136851171804647584820917879605413346513247323262592357770641866841786782372663]
[[0, [1, 2]], [1, [0, 2]], [2, [0, 1]]]
[[ 0.5914727922595024612132584494125

In [11]:
print(emb)

[Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> 2^(1/5), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> 1/4*2^(1/5)*(sqrt(5) + I*sqrt(2*sqrt(5) + 10) - 1), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> -1/4*2^(1/5)*(sqrt(5) - I*sqrt(-2*sqrt(5) + 10) + 1), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> 1/4*2^(1/5)*(sqrt(5) - I*sqrt(2*sqrt(5) + 10) - 1), Generic morphism:
  From: Number Field in theta with defining polynomial x^5 - 2
  To:   Symbolic Ring
  Defn: theta -> -1/4*2^(1/5)*(sqrt(5) + I*conjugate(sqrt(-2*sqrt(5) + 10)) + 1)]


In [12]:
K.<theta> = NumberField(poly) ### Redefine theta as in the number field, otherwise it's recognised as in the symbolic ring

import numpy

delta=[sum(numpy.abs(V[i].row(j).list())) for i in range(rk+1) for j in range(rk)]
print(delta)

u=norm_list[0]
LL=[[ln( abs(  ((emb[II[i][0]](theta)- emb[j](theta))/emb[j](u)) )   ) for j in II[i][1] ] for i in range(rk+1)]

print(LL)

LV=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]
for i in range(rk):
    for j in range(rk+1):
        LV[j].set_col_to_multiple_of_col(i, i, LL[j][i])

print(LV)

[0.788360261943747302709157813058308708970918541387650452598628901089285246169194539331375841609067765123312417790315675662910505653697070475964794281494, 2.22452522055574512869523988475173611457336618752957351122901219922232411743634403638047582843817252532870011913784559494787067801370461287158262617189, 1.08409665807349853331578703776457528243691402823779656972508602298392470678269718062546605399185902170296454066123005319010548299671840963392554757005, 0.870079035929497218321471720105237887942895334724975471470944518476405466999292693032150981711540514330074922001310602244866069714873105978916784601539, 0.886803996785869723457107494881196489990382325231329561771486229739288408640289167702973550030810391105316503814559135967170781137524266555986438133684, 0.744499943613272843862407020538316344223073819587942634005467931008075691943210019457338171870741561130914965114570920841706709630294324874508333378113]
[[log(abs(-1/4*2^(1/5)*(sqrt(5) + I*sqrt(2*sqrt(5) + 10) - 1) + 2^(1/5))), lo

In [13]:
### Calculating c9, c10, c11, lamdba_k's

lam=[sum(LV[i].row(j)) for i in range(rk+1) for j in range(rk)]
print(lam)
c10=max(abs(delta[0]),abs(delta[1]))
c11=max(((abs(V[0][0,0])+abs(V[0][0,1]))/deg )+abs(lam[0]),  ((abs(V[0][1,0])+abs(V[0][1,1]))/deg )+abs(lam[1]))
c9=max([2^(deg-1)*abs(m)/abs(poly.derivative()(thetas[i])) for i in range(deg)])
print('c9=',c9)
print('c10=',c10)
print('c11=',c11)

M=matrix(CC,deg,deg)
for i in range(deg):
    for j in range(deg):
        if j < i:
            M[i,j] = abs(thetas[i]-thetas[j])
        else:
            M[i,j] = abs(thetas[i])+abs(thetas[j])


c1=2/min(M.list())

print(c1)
c14=c1*c9*e^(deg*c11/c10)
c15=deg/c10
print('c14=',c14)
print('c15=',c15)
c16=c14*(V[0].norm(Infinity))
print('c16=',c16)

[0.0237821450001311139759770304458648558212187705070000854197760040764986078235220376779644992110187563587844821443932503538208724909087467407102145113836, 1.14301333952178357892913793214606634846966760833744120980613528237284899942732664146869890946248266234680628219894074158402734022049214645975020214997, -0.325644208274987406994168764943065567962964918938335496138185899967923349442034190511654943638891350025547664708688289622949494741107112145639412825262, -0.261356952520663605612355667050410422604244194308259997701296411351241524637715490218557451636816338653553228934616092988524683383593239061318184918694, 0.313753135774921850006180249720133140052355533684835453428297897929674045530273171672672694033381971846155423636491664446039058495652738775284305569571, -0.310149717240228183852213299022622751630589609860460607201771229835182975075947830515792003094424992519849912164854277803488986726652834168556916156289]


c9= 8/5*2^(1/5)
c10= 2.22452522055574512869523988475173611457336618752957351122901219922232411743634403638047582843817252532870011913784559494787067801370461287158262617189
c11= 1.58791838363293260466818590909641357138434084584335591205193772221731382291459544874479407515011716741254630602650986057360147582323306903406672738435
1.48106908075834
c14= 84.0898036659179*2^(1/5)
c15= 2.24767062822999507168986409914195677945661690758690190274621771243750846714061155716423563798984866098303641038167752963093661865529375335626054837386


c16= 187.059889046415*2^(1/5)


In [14]:
### Calculating S_g, T_g

delta_h=max(abs(delta[0]),abs(delta[1]))
print('delta_h=',delta_h)
h=0
if delta_h in delta == True:
    h=delta.index(delta_h)
else:
    if -delta_h in delta == True:
        h=delta.index(-delta_h)
    
if h == 0 :
    g=1
else:
    g=0
    
S_g=delta[g]/delta[h]
T_g=(delta[g]*lam[h]-delta[h]*lam[g])/delta[h]

print('S_g=',S_g)
print('T_g=',T_g)

delta_h= 2.22452522055574512869523988475173611457336618752957351122901219922232411743634403638047582843817252532870011913784559494787067801370461287158262617189
S_g= 2.82171150416822253755517409498274983772517800074424393303623874195800650558491334580466104615797012574145682678823701610235349949294168228702195408962
T_g= -1.07590698738111684033436121354433990223184431784600043217625207888247016827258044570465813192930327787896958010105425226162090854459422938261774719385


In [17]:
### Funtion to find q as in the text, using continued fractions
def cfreduction(S_g,A_0,k):
    X0=k*A_0
    index=0
    c=continued_fraction(S_g)
    while abs(c.numerator(index)-(S_g)*(c.denominator(index))) > 1/X0:
        index=index+1
        if c.denominator(index) > X0:
            break
    q=c.denominator(index)
    return(q,index)
A_0=1.26*10^26
k=50
q=cfreduction(S_g,A_0,k)[0]
index=cfreduction(S_g,A_0,k)[1]
print(q)
print(index)

1211146255498931339729838419
66


In [19]:
print(continued_fraction(S_g).numerator(index))
print(abs(continued_fraction(S_g).numerator(index)-(S_g)*(continued_fraction(S_g).denominator(index))))

3417505322371599917472294776
6.58304222175076929910984040102074138477050509484554751257593033252623547102162616180949096398466041314727272802076791727171219158922946944052216417535e-29


In [21]:
RR500=RealField(500)
dist=min(  RR500(q*(T_g)).ceil()-q*(T_g),  q*(T_g)-RR500(q*(T_g)).floor()  )   ### distance to neareast integer of q*T_g
while dist < 2/k:
    k=2*k
    q=cfreduction(S_g,A_0,k)
    if k > 10000:
        break
new_bound=RR((1/c15)*ln(2*c16*k^2*A_0)).ceil() 
print(new_bound)

33


In [24]:
print(dist)

0.273758137777587549938001315255400374914397095936247362717733274271162456393532680089666124182720971303935604674232035042560581620662805448199780705143


In [25]:
XY=[]
for i in range(-new_bound,new_bound):
    for j in range(-new_bound,new_bound):
        for k in range(len(norm_list)):
            result=(norm_list[k])*(FU[0]^i)*(FU[1]^j)
            P=result.polynomial(var='t')
            if P.degree() <2:
                a0=abs(result[0])
                a1=abs(result[1])
                XY.append((a0,a1))
print(sorted(set(XY)))

[(1, 0), (1, 1)]
